<a href="https://colab.research.google.com/github/alejandrohs4733/Estudio/blob/main/Red%20neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import zipfile
import requests
import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout

# Cargar los datos
url = "http://archive.ics.uci.edu/static/public/262/turkiye+student+evaluation.zip"
almacenamiento = requests.get(url)

# Verificar descarga
if almacenamiento.status_code == 200:
    # Abriendo el archivo .zip desde la respuesta
    with zipfile.ZipFile(io.BytesIO(almacenamiento.content)) as archivo_zip:
        # Listando archivos
        print("Archivos en el ZIP:", archivo_zip.namelist())

        # Extrayendo archivo
        nombre_csv = archivo_zip.namelist()[0]  # Primer archivo
        with archivo_zip.open(nombre_csv) as archivo_csv:
            # Cargar .csv en dataframe pandas
            df = pd.read_csv(archivo_csv, sep=',')
            print(df.head())  # Primeras filas del dataframe
else:
    print("Error en la descarga", almacenamiento.status_code)

data = df

# Preprocesamiento de datos
X = data.drop(columns=['Q28'])
y = data['Q28']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Construcción de la red neuronal
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compilación del modelo
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)

# Evaluación del modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Pérdida: {loss:.4f}, Precisión: {accuracy:.4f}')



In [ ]:
plt.figure(figsize=(12, 4))

# Gráfico de la pérdida durante el entrenamiento
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.title('Pérdida durante el entrenamiento')

# Gráfico de MAE (Mean Absolute Error) durante el entrenamiento
plt.subplot(1, 2, 2)
plt.plot(history.history['mean_absolute_error'], label='MAE de entrenamiento')
plt.plot(history.history['val_mean_absolute_error'], label='MAE de validación')
plt.xlabel('Épocas')
plt.ylabel('MAE')
plt.legend()
plt.title('MAE durante el entrenamiento')

plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print(f'Error Cuadratico Medio: {mse}')

r2 = r2_score(y_test, predictions)
print(f'Coeficiente de Determinacion: {r2}')
